# CS 520 Data Curation Project

## Our goal for the project is to Extract data from a CSV file. 
## Transform the original schema of the file into the required schema with lesser number of columns and with column names changed. 
## Explore and Query the Data. 
## Save the data as JSON Documents in a MapR-DB JSON Table. 
## All this is done using Spark in Python.



## We have used the Medicare Open payments data from a CSV file.

## Importing SparkSession

In [1]:
from pyspark.sql import SparkSession

## Building SparkSession 

In [2]:
spark = SparkSession.builder.appName('CS 520').getOrCreate()

## reading the csv File 

In [24]:
df = spark.read.csv("payments.csv", header = True)

## Print the original schema 

In [49]:
df.printSchema()

root
 |-- Change_Type: string (nullable = true)
 |-- Covered_Recipient_Type: string (nullable = true)
 |-- Teaching_Hospital_CCN: string (nullable = true)
 |-- Teaching_Hospital_ID: string (nullable = true)
 |-- Teaching_Hospital_Name: string (nullable = true)
 |-- Physician_Profile_ID: string (nullable = true)
 |-- Physician_First_Name: string (nullable = true)
 |-- Physician_Middle_Name: string (nullable = true)
 |-- Physician_Last_Name: string (nullable = true)
 |-- Physician_Name_Suffix: string (nullable = true)
 |-- Recipient_Primary_Business_Street_Address_Line1: string (nullable = true)
 |-- Recipient_Primary_Business_Street_Address_Line2: string (nullable = true)
 |-- Recipient_City: string (nullable = true)
 |-- Recipient_State: string (nullable = true)
 |-- Recipient_Zip_Code: string (nullable = true)
 |-- Recipient_Country: string (nullable = true)
 |-- Recipient_Province: string (nullable = true)
 |-- Recipient_Postal_Code: string (nullable = true)
 |-- Physician_Primary_Ty

## Changing the data type of Amount from String to Double  

In [25]:
from pyspark.sql.types import DoubleType


In [26]:
df2 = df.withColumn("amount" , df["Total_Amount_of_Payment_USDollars"].cast(DoubleType()))

## Creating a Temporary Payments 

In [ ]:
df2.createGlobalTempView("payments1")

## We can also specify the schema while importing the file in the below manner 

In [28]:
from pyspark.sql.types import StructField,StringType,IntegerType,StructType

In [29]:
data_schema = [StructField("physician_id", StringType(), True),StructField("date_payment", StringType(), True),StructField("record_id", StringType(), True),StructField("payer", StringType(), True),StructField("amount", DoubleType(), True),StructField("physician_speciality", StringType(), True),StructField("nature_of_payment", StringType(), True)]

In [30]:
final_struc = StructType(fields=data_schema)

## Selecting only the columns we want and also renaming the Columns as we want 

In [35]:
ds = spark.sql("select Physician_Profile_ID as physician_id,Date_of_Payment as date_payment, Record_ID as record_id, Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name as payer,amount, Physician_Specialty, Nature_of_Payment_or_Transfer_of_Value as Nature_of_payment from global_temp.payments1 where Physician_Profile_ID IS NOT NULL") 

## Required Schema 

In [37]:
ds.printSchema()

root
 |-- physician_id: string (nullable = true)
 |-- date_payment: string (nullable = true)
 |-- record_id: string (nullable = true)
 |-- payer: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- Physician_Specialty: string (nullable = true)
 |-- Nature_of_payment: string (nullable = true)



In [42]:
ds.cache()

DataFrame[physician_id: string, date_payment: string, record_id: string, payer: string, amount: double, Physician_Specialty: string, Nature_of_payment: string]

In [43]:
ds.first()

Row(physician_id='673985', date_payment='01/21/2016', record_id='346039414', payer='DFINE, Inc', amount=286.2, Physician_Specialty='Allopathic & Osteopathic Physicians|Anesthesiology', Nature_of_payment='Travel and Lodging')

## Replacing the temporary view with our new view 

In [44]:
ds.createOrReplaceGlobalTempView("payments")

## Sample data 

In [45]:
ds.show()

+------------+------------+---------+----------+------+--------------------+------------------+
|physician_id|date_payment|record_id|     payer|amount| Physician_Specialty| Nature_of_payment|
+------------+------------+---------+----------+------+--------------------+------------------+
|      673985|  01/21/2016|346039414|DFINE, Inc| 286.2|Allopathic & Oste...|Travel and Lodging|
|      673985|  01/21/2016|346039416|DFINE, Inc|  25.0|Allopathic & Oste...|Travel and Lodging|
|      673985|  02/19/2016|346039418|DFINE, Inc| 27.27|Allopathic & Oste...|Travel and Lodging|
|       93975|  04/15/2016|346039420|DFINE, Inc|  21.6|Allopathic & Oste...| Food and Beverage|
|      275444|  04/29/2016|346039422|DFINE, Inc| 22.57|Allopathic & Oste...| Food and Beverage|
|      132655|  02/05/2016|346039424|DFINE, Inc| 780.7|Allopathic & Oste...|Travel and Lodging|
|      132655|  02/05/2016|346039426|DFINE, Inc|  25.0|Allopathic & Oste...|Travel and Lodging|
|      132655|  02/19/2016|346039428|DFI

## Querying and Exploring the data  

## Querying can be done in two ways. One by using Spark functions and other directly by wrinting SQl statements. Both the ways are used below. 

### Top 10 nature of accounts with payments by count  

In [64]:
from pyspark.sql.functions import desc
ds.groupBy(ds["Nature_of_Payment"]).count().orderBy(desc("count")).show(10)

+--------------------+-------+
|   Nature_of_Payment|  count|
+--------------------+-------+
|   Food and Beverage|9975364|
|  Travel and Lodging| 575255|
|Compensation for ...| 251440|
|           Education| 245515|
|      Consulting Fee| 115316|
|                Gift|  41523|
|           Honoraria|  17795|
|  Royalty or License|  11814|
|Compensation for ...|   9171|
|       Entertainment|   8408|
+--------------------+-------+
only showing top 10 rows



### Nature of payments with payments  > $1000 with their counts

In [61]:
ds.filter(ds["amount"] > 1000).groupBy(ds["Nature_of_Payment"]).count().show()

+--------------------+------+
|   Nature_of_Payment| count|
+--------------------+------+
|           Education|  8378|
|       Entertainment|    30|
|  Travel and Lodging| 24196|
|Charitable Contri...|   235|
|Current or prospe...|   523|
|           Honoraria| 13052|
|               Grant|  2154|
|Compensation for ...|  2032|
|  Royalty or License|  9151|
|Compensation for ...|  5688|
|   Food and Beverage|   430|
|Compensation for ...|190064|
|      Consulting Fee| 71516|
|                Gift|  1234|
+--------------------+------+



### Top five Physicain specialites by total amount 

In [65]:
from pyspark.sql.functions import sum

In [76]:
spark.sql ("select physician_id , sum(amount) as revenue from global_thttps://mapr.com/blog/etl-pipeline-healthcare-dataset-with-spark-json-mapr-db/assets/queries-chart-example-3.pngemp.payments group by physician_id order by revenue desc limit 5").show() 

+------------+--------------------+
|physician_id|             revenue|
+------------+--------------------+
|      288926|       2.183833534E7|
|      311622|1.9940975750000004E7|
|      327184|1.3732734540000003E7|
|      281659|         1.3202939E7|
|       32719|1.2149519940000001E7|
+------------+--------------------+



### Top 10  nature of payments by total amount

In [78]:
spark.sql("select Nature_of_payment , sum(amount) as total from global_temp.payments group by Nature_of_payment order by total desc limit 10").show()

+--------------------+--------------------+
|   Nature_of_payment|               total|
+--------------------+--------------------+
|Compensation for ...| 5.618398276899997E8|
|  Royalty or License| 4.892801498499998E8|
|      Consulting Fee|3.6266093983000004E8|
|   Food and Beverage|2.4166747336000344E8|
|  Travel and Lodging|1.9028856042000213E8|
|Current or prospe...|6.2862144450000025E7|
|           Honoraria|       4.392361441E7|
|           Education| 3.895051414999967E7|
|               Grant|       2.389616791E7|
|Compensation for ...|       2.006624783E7|
+--------------------+--------------------+

